# 🧠 Modelo de Optimización Estocástica en Monopoly

Este modelo busca determinar la mejor forma de invertir $1500 en propiedades y construcciones en Monopoly, **maximizando la renta esperada**, bajo restricciones del juego.

---

## 🎯 Objetivo

**Maximizar los ingresos esperados por alquiler**, considerando:

- Probabilidad de caer en cada casilla (simplificada como uniforme).
- Decisiones de compra y construcción (casas y hoteles).
- Presupuesto limitado.

---

## 🧱 Variables del Modelo

| Variable | Tipo | Descripción |
|---------|------|-------------|
| `x[i]` | Binaria | Comprar la propiedad `i` |
| `y_n[i]` | Binaria | Construir `n` casas (1 a 4) en la propiedad `i` |
| `z[i]` | Binaria | Construir hotel en la propiedad `i` |
| `g[color]` | Binaria | Monopolio del grupo de color `color` |

---

## 🧮 Función Objetivo

Maximizar la **renta esperada**:

\[
\text{Max } \sum_{i} \text{Probabilidad}(i) \times \text{Renta}_i
\]

Donde la renta incluye:

- Alquiler base (`x[i]`).
- Alquiler con 1-4 casas (`y_n[i]`).
- Alquiler con hotel (`z[i]`).

---

## 💸 Restricción de Presupuesto

Costo total ≤ dinero disponible (`$1500`):

\[
\sum \text{Compra} + \sum \text{Casas} + \sum \text{Hoteles} \leq 1500
\]

---

## 📏 Restricciones del Juego

- **Construcción solo si se compra la propiedad**:
  ```python
  y_n[i] <= x[i], z[i] <= x[i]


In [6]:
class Casilla:
    def __init__(self, nombre, tipo, costo=0, renta=0, costo_casa=0, renta_con_casas=None, costo_hotel=0, renta_hotel=0, grupo=None):
        self.nombre = nombre
        self.tipo = tipo
        self.costo = costo
        self.renta = renta
        self.propietario = None
        self.casas = 0
        self.hoteles = 0
        self.costo_casa = costo_casa
        self.costo_hotel = costo_hotel or costo_casa  # Default to costo_casa if not specified
        self.renta_con_casas = renta_con_casas or [0] * 4  # Explicit rents for 1-4 houses
        self.renta_hotel = renta_hotel
        self.grupo = grupo
        self.hipotecada = False
        self.valor_hipoteca = costo // 2 if costo > 0 else 0

    def calcular_renta(self, tablero):
        if self.hipotecada:
            return 0
        if self.hoteles > 0:
            return self.renta_hotel
        elif self.casas > 0:
            return self.renta_con_casas[self.casas - 1]
        else:
            # Double rent for monopoly
            if self.grupo and self.propietario:
                grupo = [p for p in tablero.casillas if p.grupo == self.grupo and p.tipo == "propiedad"]
                if all(p.propietario == self.propietario for p in grupo):
                    return self.renta * 2
            return self.renta

In [7]:
class Tablero:
    def __init__(self):
        print("Creando tablero...")
        self.casillas = self.crear_tablero()
        self.grupos = self.crear_grupos()
        self.posicion_carcel = next(i for i, c in enumerate(self.casillas) if c.nombre == "Cárcel / Solo de visita")

    def crear_grupos(self):
        grupos = {}
        for casilla in self.casillas:
            if casilla.grupo:
                if casilla.grupo not in grupos:
                    grupos[casilla.grupo] = []
                grupos[casilla.grupo].append(casilla)
        return grupos

    def crear_tablero(self):
        print("Inicializando las casillas...")
        return [
            Casilla("Salida", "especial"),
            Casilla("Mediterranean Avenue", "propiedad", costo=60, renta=2, costo_casa=50, costo_hotel=50, renta_con_casas=[10, 30, 90, 160], renta_hotel=250, grupo="morado"),
            Casilla("Comunidad", "comunidad"),
            Casilla("Baltic Avenue", "propiedad", costo=60, renta=4, costo_casa=50, costo_hotel=50, renta_con_casas=[20, 60, 180, 320], renta_hotel=450, grupo="morado"),
            Casilla("Impuesto sobre la renta", "impuesto", costo=200),
            Casilla("Ferrocarril Reading", "ferrocarril", costo=200, renta=25),
            Casilla("Oriental Avenue", "propiedad", costo=100, renta=6, costo_casa=50, costo_hotel=50, renta_con_casas=[30, 90, 270, 400], renta_hotel=550, grupo="celeste"),
            Casilla("Suerte", "suerte"),
            Casilla("Vermont Avenue", "propiedad", costo=100, renta=6, costo_casa=50, costo_hotel=50, renta_con_casas=[30, 90, 270, 400], renta_hotel=550, grupo="celeste"),
            Casilla("Connecticut Avenue", "propiedad", costo=120, renta=8, costo_casa=50, costo_hotel=50, renta_con_casas=[40, 100, 300, 450], renta_hotel=600, grupo="celeste"),
            Casilla("Cárcel / Solo de visita", "especial"),
            Casilla("St. Charles Place", "propiedad", costo=140, renta=10, costo_casa=100, costo_hotel=100, renta_con_casas=[50, 150, 450, 625], renta_hotel=750, grupo="rosado"),
            Casilla("Comunidad", "comunidad"),
            Casilla("States Avenue", "propiedad", costo=140, renta=10, costo_casa=100, costo_hotel=100, renta_con_casas=[50, 150, 450, 625], renta_hotel=750, grupo="rosado"),
            Casilla("Virginia Avenue", "propiedad", costo=160, renta=12, costo_casa=100, costo_hotel=100, renta_con_casas=[60, 180, 500, 700], renta_hotel=900, grupo="rosado"),
            Casilla("Ferrocarril Pennsylvania", "ferrocarril", costo=200, renta=25),
            Casilla("St. James Place", "propiedad", costo=180, renta=14, costo_casa=100, costo_hotel=100, renta_con_casas=[70, 200, 550, 750], renta_hotel=950, grupo="naranja"),
            Casilla("Suerte", "suerte"),
            Casilla("Tennessee Avenue", "propiedad", costo=180, renta=14, costo_casa=100, costo_hotel=100, renta_con_casas=[70, 200, 550, 750], renta_hotel=950, grupo="naranja"),
            Casilla("New York Avenue", "propiedad", costo=200, renta=16, costo_casa=100, costo_hotel=100, renta_con_casas=[80, 220, 600, 800], renta_hotel=1000, grupo="naranja"),
            Casilla("Parada libre", "especial"),
            Casilla("Kentucky Avenue", "propiedad", costo=220, renta=18, costo_casa=150, costo_hotel=150, renta_con_casas=[90, 250, 700, 875], renta_hotel=1050, grupo="rojo"),
            Casilla("Suerte", "suerte"),
            Casilla("Indiana Avenue", "propiedad", costo=220, renta=18, costo_casa=150, costo_hotel=150, renta_con_casas=[90, 250, 700, 875], renta_hotel=1050, grupo="rojo"),
            Casilla("Illinois Avenue", "propiedad", costo=240, renta=20, costo_casa=150, costo_hotel=150, renta_con_casas=[100, 300, 750, 925], renta_hotel=1100, grupo="rojo"),
            Casilla("Ferrocarril B&O", "ferrocarril", costo=200, renta=25),
            Casilla("Atlantic Avenue", "propiedad", costo=260, renta=22, costo_casa=150, costo_hotel=150, renta_con_casas=[110, 330, 800, 975], renta_hotel=1150, grupo="amarillo"),
            Casilla("Ventnor Avenue", "propiedad", costo=260, renta=22, costo_casa=150, costo_hotel=150, renta_con_casas=[110, 330, 800, 975], renta_hotel=1150, grupo="amarillo"),
            Casilla("Compañía de agua", "servicio", costo=150, renta=0),
            Casilla("Marvin Gardens", "propiedad", costo=280, renta=24, costo_casa=150, costo_hotel=150, renta_con_casas=[120, 360, 850, 1025], renta_hotel=1200, grupo="amarillo"),
            Casilla("Ve a la cárcel", "especial"),
            Casilla("Pacific Avenue", "propiedad", costo=300, renta=26, costo_casa=200, costo_hotel=200, renta_con_casas=[130, 390, 900, 1100], renta_hotel=1275, grupo="verde"),
            Casilla("Carolina del Norte Avenue", "propiedad", costo=300, renta=26, costo_casa=200, costo_hotel=200, renta_con_casas=[130, 390, 900, 1100], renta_hotel=1275, grupo="verde"),
            Casilla("Comunidad", "comunidad"),
            Casilla("Avenida Pennsylvania", "propiedad", costo=320, renta=28, costo_casa=200, costo_hotel=200, renta_con_casas=[150, 450, 1000, 1200], renta_hotel=1400, grupo="verde"),
            Casilla("Ferrocarril Short Line", "ferrocarril", costo=200, renta=25),
            Casilla("Suerte", "suerte"),
            Casilla("Park Place", "propiedad", costo=350, renta=35, costo_casa=200, costo_hotel=200, renta_con_casas=[175, 500, 1100, 1300], renta_hotel=1500, grupo="azul oscuro"),
            Casilla("Impuesto de lujo", "impuesto", costo=100),
            Casilla("Boardwalk", "propiedad", costo=400, renta=50, costo_casa=200, costo_hotel=200, renta_con_casas=[200, 600, 1400, 1700], renta_hotel=2000, grupo="azul oscuro")
        ]

In [8]:
import pulp
import numpy as np

def calcular_matriz_transicion(tablero):
    n = len(tablero.casillas)
    P = np.zeros((n, n))
    probs_dados = {2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36, 
                   8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36}
    for i in range(n):
        for suma, prob in probs_dados.items():
            nueva_pos = (i + suma) % n
            if tablero.casillas[nueva_pos].nombre == "Ve a la cárcel":
                P[i, tablero.posicion_carcel] += prob
            else:
                P[i, nueva_pos] += prob
    P[tablero.posicion_carcel, tablero.posicion_carcel] = 0.5
    for suma, prob in probs_dados.items():
        if suma in [2, 4, 6, 8, 10, 12]:
            P[tablero.posicion_carcel, (tablero.posicion_carcel + suma) % n] += prob / 2
    return P

def calcular_estacionaria(P):
    n = P.shape[0]
    A = P.T - np.eye(n)
    A[-1] = np.ones(n)
    b = np.zeros(n)
    b[-1] = 1
    pi = np.linalg.solve(A, b)
    return pi

In [9]:
def optimizacion_estocastica_monopoly(tablero, dinero_total=1500):
    propiedades = [c for c in tablero.casillas if c.tipo == "propiedad"]
    num_propiedades = len(propiedades)
    grupos = tablero.grupos
    indices = [i for i, c in enumerate(tablero.casillas) if c.tipo == "propiedad"]

    # Calcular distribución estacionaria (simplificada, basada en dados)
    n = len(tablero.casillas)
    estacionaria = [1/n for _ in range(n)]  # Aproximación uniforme para simplificar

    # Crear el modelo
    modelo = pulp.LpProblem("Optimizacion_Monopoly", pulp.LpMaximize)

    # Variables
    x = {i: pulp.LpVariable(f"x_{i}", cat="Binary") for i in range(num_propiedades)}  # Comprar propiedad
    y_1 = {i: pulp.LpVariable(f"y_1_{i}", cat="Binary") for i in range(num_propiedades)}  # 1 casa
    y_2 = {i: pulp.LpVariable(f"y_2_{i}", cat="Binary") for i in range(num_propiedades)}  # 2 casas
    y_3 = {i: pulp.LpVariable(f"y_3_{i}", cat="Binary") for i in range(num_propiedades)}  # 3 casas
    y_4 = {i: pulp.LpVariable(f"y_4_{i}", cat="Binary") for i in range(num_propiedades)}  # 4 casas
    z = {i: pulp.LpVariable(f"z_{i}", cat="Binary") for i in range(num_propiedades)}  # Hotel
    g = {color: pulp.LpVariable(f"g_{color}", cat="Binary") for color in grupos}  # Monopolio

    # Objetivo: Maximizar renta esperada
    renta_esperada = pulp.lpSum([
        estacionaria[indices[i]] * (
            propiedades[i].renta * x[i] +  # Base rent
            propiedades[i].renta_con_casas[0] * y_1[i] +  # Rent with 1 house
            propiedades[i].renta_con_casas[1] * y_2[i] +  # Rent with 2 houses
            propiedades[i].renta_con_casas[2] * y_3[i] +  # Rent with 3 houses
            propiedades[i].renta_con_casas[3] * y_4[i] +  # Rent with 4 houses
            propiedades[i].renta_hotel * z[i]  # Rent with hotel
        ) for i in range(num_propiedades)
    ])
    modelo += renta_esperada

    # Costo total
    costo_total = (
        pulp.lpSum([propiedades[i].costo * x[i] for i in range(num_propiedades)]) +
        pulp.lpSum([propiedades[i].costo_casa * (y_1[i] + 2*y_2[i] + 3*y_3[i] + 4*y_4[i]) for i in range(num_propiedades)]) +
        pulp.lpSum([propiedades[i].costo_hotel * z[i] for i in range(num_propiedades)])
    )
    modelo += costo_total <= dinero_total

    # Restricciones
    for i in range(num_propiedades):
        # No se pueden construir casas ni hoteles sin comprar la propiedad
        modelo += y_1[i] <= x[i]
        modelo += y_2[i] <= x[i]
        modelo += y_3[i] <= x[i]
        modelo += y_4[i] <= x[i]
        modelo += z[i] <= x[i]
        # Solo un nivel de construcción por propiedad
        modelo += y_1[i] + y_2[i] + y_3[i] + y_4[i] + z[i] <= 1
        # Hotel requiere monopolio y 4 casas
        modelo += z[i] <= y_4[i]

    # Monopolios
    for color, casillas in grupos.items():
        indices_grupo = [i for i, c in enumerate(propiedades) if c.grupo == color]
        # Si todas las propiedades del grupo están compradas, g[color] = 1
        modelo += len(indices_grupo) * g[color] <= pulp.lpSum([x[i] for i in indices_grupo])
        for i in indices_grupo:
            # Construcción requiere monopolio
            modelo += y_1[i] <= g[color]
            modelo += y_2[i] <= g[color]
            modelo += y_3[i] <= g[color]
            modelo += y_4[i] <= g[color]
            modelo += z[i] <= g[color]

    # Resolver
    modelo.solve()
    resultados = []
    for i in range(num_propiedades):
        casas = 1 if y_1[i].value() == 1 else 2 if y_2[i].value() == 1 else 3 if y_3[i].value() == 1 else 4 if y_4[i].value() == 1 else 0
        hotel = z[i].value() == 1
        if x[i].value() == 1:
            resultados.append({
                "propiedad": propiedades[i].nombre,
                "casas": casas,
                "hotel": hotel
            })
    return resultados

In [10]:
tablero = Tablero()
resultados = optimizacion_estocastica_monopoly(tablero, dinero_total=1500)
print("\nEstrategia óptima:")
for r in resultados:
    print(f"Propiedad: {r['propiedad']}, Casas: {r['casas']}, Hotel: {r['hotel']}")

Creando tablero...
Inicializando las casillas...



Estrategia óptima:
Propiedad: Mediterranean Avenue, Casas: 4, Hotel: False
Propiedad: Baltic Avenue, Casas: 4, Hotel: False
Propiedad: Oriental Avenue, Casas: 4, Hotel: False
Propiedad: Vermont Avenue, Casas: 4, Hotel: False
Propiedad: Connecticut Avenue, Casas: 4, Hotel: False


Modelo 2 

In [11]:
import random
import math
import copy
from typing import List, Dict

def optimizacion_estocastica_monopoly_stressed(tablero, dinero_total=1200, max_iterations=1000, initial_temp=1000, cooling_rate=0.995):
    """
    Optimiza la compra y desarrollo de propiedades con restricciones de monopolio y costos estocásticos
    usando recocido simulado (simulated annealing).
    
    Parameters:
    - tablero: Instancia de Tablero con casillas.
    - dinero_total: Presupuesto inicial (reducido para estrés, default 1200).
    - max_iterations: Iteraciones máximas para el algoritmo.
    - initial_temp: Temperatura inicial para recocido simulado.
    - cooling_rate: Factor de enfriamiento por iteración.
    
    Returns:
    - Dict con la mejor configuración de propiedades, casas y hoteles.
    """
    # Extraer propiedades y grupos
    propiedades = [c for c in tablero.casillas if c.tipo == "propiedad"]
    num_propiedades = len(propiedades)
    indices = [i for i, c in enumerate(tablero.casillas) if c.tipo == "propiedad"]
    grupos = tablero.grupos

    # Distribución estacionaria aproximada (uniforme para simplificar)
    n = len(tablero.casillas)
    estacionaria = [1/n for _ in range(n)]

    # Función para calcular el costo total con variación estocástica
    def calcular_costo(solucion, aplicar_variacion=True):
        costo = 0
        for i, prop in enumerate(solucion["propiedades"]):
            if prop["comprada"]:
                costo += propiedades[i].costo
                num_casas = prop["casas"]
                if num_casas > 0:
                    costo_casas = propiedades[i].costo_casa * num_casas
                    if aplicar_variacion:
                        # Variación estocástica: ±10% en costo de casas
                        costo_casas *= random.uniform(0.9, 1.1)
                    costo += costo_casas
                if prop["hotel"]:
                    costo_hotel = propiedades[i].costo_hotel
                    if aplicar_variacion:
                        costo_hotel *= random.uniform(0.9, 1.1)
                    costo += costo_hotel
        return costo

    # Función para calcular la renta esperada
    def calcular_renta_esperada(solucion):
        renta_total = 0
        for i, prop in enumerate(solucion["propiedades"]):
            if not prop["comprada"]:
                continue
            idx = indices[i]
            base_renta = propiedades[i].renta
            # Verificar monopolio
            grupo = propiedades[i].grupo
            props_grupo = [p for p in propiedades if p.grupo == grupo]
            num_compradas_grupo = sum(1 for j, p in enumerate(propiedades) if p.grupo == grupo and solucion["propiedades"][j]["comprada"])
            es_monopolio = num_compradas_grupo == len(props_grupo)
            if es_monopolio and prop["casas"] == 0 and not prop["hotel"]:
                base_renta *= 2  # Doble renta por monopolio
            # Renta por casas (hasta 2 sin monopolio, 4 con monopolio)
            num_casas = prop["casas"]
            if num_casas > 0:
                if num_casas <= 2 or es_monopolio:
                    renta_total += estacionaria[idx] * propiedades[i].renta_con_casas[num_casas - 1]
                else:
                    renta_total += estacionaria[idx] * propiedades[i].renta_con_casas[1]  # Límite a 2 casas
            elif prop["hotel"] and es_monopolio:
                renta_total += estacionaria[idx] * propiedades[i].renta_hotel
            else:
                renta_total += estacionaria[idx] * base_renta
        # Penalizar monopolios completos para añadir estrés
        for grupo in grupos:
            num_compradas = sum(1 for j, p in enumerate(propiedades) if p.grupo == grupo and solucion["propiedades"][j]["comprada"])
            if num_compradas == len(grupos[grupo]):
                renta_total -= 50  # Penalización por monopolio completo
        return renta_total

    # Generar solución inicial
    def generar_solucion_inicial():
        solucion = {
            "propiedades": [{"comprada": False, "casas": 0, "hotel": False} for _ in range(num_propiedades)]
        }
        # Comprar algunas propiedades aleatoriamente dentro del presupuesto
        presupuesto = dinero_total
        indices_aleatorios = random.sample(range(num_propiedades), num_propiedades)
        for i in indices_aleatorios:
            if presupuesto >= propiedades[i].costo:
                solucion["propiedades"][i]["comprada"] = True
                presupuesto -= propiedades[i].costo
                # Añadir hasta 2 casas si es viable
                grupo = propiedades[i].grupo
                num_en_grupo = sum(1 for j, p in enumerate(propiedades) if p.grupo == grupo and solucion["propiedades"][j]["comprada"])
                max_casas = 2 if num_en_grupo < len(grupos[grupo]) else 4
                casas_posibles = min(max_casas, presupuesto // propiedades[i].costo_casa)
                solucion["propiedades"][i]["casas"] = random.randint(0, casas_posibles)
                presupuesto -= solucion["propiedades"][i]["casas"] * propiedades[i].costo_casa
                # Añadir hotel si hay monopolio y 4 casas
                if num_en_grupo == len(grupos[grupo]) and solucion["propiedades"][i]["casas"] == 4 and presupuesto >= propiedades[i].costo_hotel:
                    solucion["propiedades"][i]["hotel"] = True
                    solucion["propiedades"][i]["casas"] = 0
                    presupuesto -= propiedades[i].costo_hotel
        return solucion

    # Generar vecino
    def generar_vecino(solucion_actual):
        vecino = copy.deepcopy(solucion_actual)
        i = random.randint(0, num_propiedades - 1)
        prop = vecino["propiedades"][i]
        grupo = propiedades[i].grupo
        num_en_grupo = sum(1 for j, p in enumerate(propiedades) if p.grupo == grupo and vecino["propiedades"][j]["comprada"])

        # Acciones posibles: comprar/vender propiedad, añadir/quitar casas, añadir/quitar hotel
        accion = random.choice(["comprar", "casas", "hotel"] if not prop["comprada"] else ["vender", "casas", "hotel"])
        
        if accion == "comprar" and not prop["comprada"]:
            # Evitar monopolio completo
            if num_en_grupo + 1 < len(grupos[grupo]):
                prop["comprada"] = True
                prop["casas"] = 0
                prop["hotel"] = False
        elif accion == "vender" and prop["comprada"]:
            prop["comprada"] = False
            prop["casas"] = 0
            prop["hotel"] = False
        elif accion == "casas" and prop["comprada"]:
            max_casas = 2 if num_en_grupo < len(grupos[grupo]) else 4
            prop["casas"] = random.randint(0, max_casas)
            prop["hotel"] = False
        elif accion == "hotel" and prop["comprada"] and num_en_grupo == len(grupos[grupo]) and prop["casas"] == 4:
            prop["hotel"] = not prop["hotel"]
            prop["casas"] = 0 if prop["hotel"] else 4

        # Verificar presupuesto
        if calcular_costo(vecino, aplicar_variacion=False) > dinero_total:
            return solucion_actual  # Rechazar si excede presupuesto
        return vecino

    # Recocido simulado
    mejor_solucion = generar_solucion_inicial()
    mejor_renta = calcular_renta_esperada(mejor_solucion)
    solucion_actual = copy.deepcopy(mejor_solucion)
    renta_actual = mejor_renta
    temperatura = initial_temp

    for _ in range(max_iterations):
        vecino = generar_vecino(solucion_actual)
        renta_vecino = calcular_renta_esperada(vecino)
        
        # Aceptar vecino según criterio de recocido simulado
        if renta_vecino > renta_actual:
            solucion_actual = vecino
            renta_actual = renta_vecino
            if renta_vecino > mejor_renta:
                mejor_solucion = vecino
                mejor_renta = renta_vecino
        else:
            delta = renta_vecino - renta_actual
            if random.random() < math.exp(delta / temperatura):
                solucion_actual = vecino
                renta_actual = renta_vecino
        
        # Enfriar temperatura
        temperatura *= cooling_rate

    # Formatear resultado
    resultados = []
    for i, prop in enumerate(mejor_solucion["propiedades"]):
        if prop["comprada"]:
            resultados.append({
                "propiedad": propiedades[i].nombre,
                "casas": prop["casas"],
                "hotel": prop["hotel"]
            })
    return {
        "resultados": resultados,
        "renta_esperada": mejor_renta
    }

In [12]:
tablero = Tablero()
resultado = optimizacion_estocastica_monopoly_stressed(tablero, dinero_total=1200)
print("\nEstrategia óptima bajo estrés:")
for r in resultado["resultados"]:
    print(f"Propiedad: {r['propiedad']}, Casas: {r['casas']}, Hotel: {r['hotel']}")
print(f"Renta esperada: {resultado['renta_esperada']:.2f}")

Creando tablero...
Inicializando las casillas...

Estrategia óptima bajo estrés:
Propiedad: States Avenue, Casas: 1, Hotel: False
Propiedad: Virginia Avenue, Casas: 0, Hotel: False
Propiedad: Park Place, Casas: 2, Hotel: False
Renta esperada: 14.05
